In [41]:
import sys 
import numpy as np 
sys.path.append('/home/kman/papers/QFPaper/Code/')
import hmm 
import pandas as pd

import seaborn as sns
from hmm import hmm_calibration
from observation_models import *
import matplotlib.pyplot as plt 
%matplotlib inline 
import copy
import ipdb 

prng = np.random.RandomState(10)
n_components = 2
startprob = prng.rand(n_components)
startprob = startprob / startprob.sum()
transmat = prng.rand(n_components, n_components) #random starting point
transmat /= np.tile(transmat.sum(axis=1)[:, np.newaxis], (1, n_components))#normalising so all entries sum up to 1

sigmas = [ .3, 0.8, 1][0:n_components]
lambdas = [0.5, 5., 0.8][0:n_components]
weights = [0, 0, 0][0:n_components]
priors = {'sigmas':[0.6, .95, 1.5][0:n_components], 'lambdas':[1, 1.5, 0.4][0:n_components], 
          'weights':[0.1, 0.2, 0.3][0:n_components]}

sample_size = 100000
#generate state and observation data from the \"right\" distribution\n",
states = hmm_calibration.sample_states(startprob, transmat, prng, sample_size=sample_size)

observation_data=ExpUniGauss.sample_data(states,lambdas, sigmas, weights, 0, rng=prng)
data_set = pd.DataFrame()
data_set['tau'] = observation_data[0]
data_set['price_change'] = observation_data[1]
#define the calibrator and model 
# update_tag = 'tpslw'

# obs_model = Exp_UniGauss(n_components, epsilon=0)
# the_hmm = hmm_calibration(obs_model, n_components, max_iter=10) 
# result = the_hmm.run_hmm(data_set, update_tag=update_tag, initialise_method_tag='uniform',priors=priors, rng=prng)

ll = 2 
sigma = 0.5

#vv = data_set.apply(lambda row: [row['tau']*2, 3*row['price_change']], axis=1) 
#vv = data_set.apply(lambda row: pd.Series({'col1':row['tau']*2, 'col2':3*row['price_change']}), axis=1) 


In [36]:
import time 
rng = np.random.RandomState(144)
start_time = time.time() 
exps = data_set['price_change'].map(lambda xx: np.exp(-xx**2))
for i in range(1000):
    sigma = rng.uniform(0,1) 
    final = exps.map(lambda xx: xx**(-.5/sigma**2)) 
    
end_time = time.time()                 
print end_time - start_time 

0.032742023468


In [39]:
start_time = time.time() 

for i in range(1000):
    sigma = rng.uniform(0,1) 
    final = data_set['price_change'].map(lambda xx: np.exp( -xx**2*(-.5/sigma**2))) 
    
end_time = time.time()                 
print end_time - start_time 

0.0481579303741


In [ ]:
step_object =result['step_calibrations'][4]
optimal_obs_ll = step_object.obs_model_.score(data_set)
complete_data_ll_optimal = step_object.complete_data_log_likelihood(optimal_obs_ll) #q-function

num_tests = 10  #number of perturbations and re-evaluation of the Q-function
values_for_lls = [] 
original_tpm = copy.deepcopy(step_object.tpm_) 

for trial in range(num_tests):
    tpm = copy.deepcopy(original_tpm) 
    #print tpm, "\n"
    row_to_change = prng.randint(0, n_components)  
    col_to_change = prng.randint(0, n_components)
    
    random_perturb = prng.uniform(low=0., high=tpm[row_to_change, col_to_change]) 
     
    tpm[row_to_change, col_to_change] -= random_perturb 
    for jj in range(n_components): 
        if jj is not col_to_change: 
            tpm[row_to_change, jj] += random_perturb/(n_components -1) 
    
       
    step_object.tpm_ = tpm
    obs_ll = step_object.obs_model_.score(data_set)
    values_for_lls.append( step_object.complete_data_log_likelihood(obs_ll)) 
    
plt.plot(values_for_lls)
plt.axhline(y=complete_data_ll_optimal)
plt.show()

  

In [ ]:
values_for_lls 

In [ ]:
complete_data_ll_optimal 